In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os
import pandas as pd
import fiddler as fdl

cassandra.__version__

'3.28.0'

In [2]:
# This secure connect bundle is autogenerated when you donwload your SCB, if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']

auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [3]:
session.set_keyspace('fiddlerai')

In [4]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [5]:
rows = session.execute('SELECT * FROM fiddler_doc_snippets_openai')
    
df_docs = rows._current_rows
#df = df.sort_values(by=['body_blob'])
df_docs

,row_id,attributes_blob,body_blob,metadata_s,vector
0,135d4d5ccc444818bec237af468e1f4a,None,ResourceLink:https://www.fiddler.ai/resources/...,None,"[-0.014165288768708706, -0.003603511257097125,..."
1,19aded6e00714d609475964d16e7103c,None,"---\ntitle: ""No Model Task Specified""\nslug: ""...",None,"[-0.0067840032279491425, -0.017058905214071274..."
2,36e022fddb01408d9c8cabbb68923dc1,None,"---\ntitle: ""client.add_model_surrogate""\nslug...",None,"[0.009862670674920082, -0.01997697539627552, 0..."
3,74a819fd82b9400ea847b6b395bcc85b,None,"---\ntitle: ""client.list_project_roles""\nslug:...",None,"[-0.016254818066954613, -0.030227787792682648,..."
4,1d3bc3e32f87496fb4dac16c9495dcf3,None,ResourceLink:https://www.fiddler.ai/resources/...,None,"[-0.003376507433131337, -0.00568545563146472, ..."
...,...,...,...,...,...
679,b631b8885b2a4647b8bb75cc3a290c61,None,"slug: ""global-explainability"" 39.08_PM.png)\n\...",None,"[0.002450974192470312, 0.0062932721339166164, ..."
680,37f85cc417db4fe8abec087a78e329ae,None,"---\ntitle: ""Textstat (beta)""\nslug: ""textstat...",None,"[-0.013476480729877949, 0.010200224816799164, ..."
681,f4793e95a9804e9e9a4c39a81a93504f,None,"---\ntitle: ""Uploading an XGBoost Model Artifa...",None,"[0.0010574640473350883, 0.012414009310305119, ..."
682,f4d94ab6c4534df087bb3ce1818c6a57,None,"---\ntitle: ""Deploying Fiddler""\nslug: ""deploy...",None,"[0.005746942479163408, -0.010771173983812332, ..."


In [ ]:
#rows = session.execute('SELECT row_id, session_id, source_docs, response, question, comment, feedback, ts FROM fiddler_chatbot_history')
# rows = session.execute('SELECT * FROM fiddler_chatbot_history')

# df = rows._current_rows
# df = df.sort_values(by=['ts'])
# df

In [ ]:
#rid = "<class 'uuid.UUID'>"
# rows = session.execute("DELETE row_id FROM fiddler_chatbot_ledger WHERE model_name= '' ")
# df = rows._current_rows
# df

In [59]:
pd.set_option('display.max_colwidth', None)

In [ ]:
rows = session.execute("SELECT * FROM fiddler_chatbot_ledger")

df = rows._current_rows
df = df.sort_values(by=['ts'], ascending=False)
df.head(5)


In [ ]:
#rows = session.execute('INSERT INTO fiddler_chatbot_ledger (row_id, session_id, comment, feedback, prompt, response, source_doc0, ts) VALUES SELECT row_id, session_id, comment, feedback, question, response, source_docs, ts FROM fiddler_chatbot_history')
rows = session.execute('SELECT row_id, session_id, comment, feedback, question, response, source_docs, ts FROM fiddler_chatbot_history')
#rows = session.execute('SELECT row_id, session_id, comment, feedback, prompt, response, source_doc0, ts FROM fiddler_chatbot_ledger')
df = rows._current_rows
df

In [44]:
# This code does the initial migration from history table to ledger table
import numpy as np

rows = session.execute('SELECT row_id, session_id, comment, feedback, question, response, source_docs, ts FROM fiddler_chatbot_history')
df = rows._current_rows

query = "INSERT INTO fiddler_chatbot_ledger (row_id, session_id, comment, feedback, prompt, response, source_doc0, ts) VALUES (?,?,?,?,?,?,?,?)"
prepared = session.prepare(query)

for index, row in df.iterrows():
    if np.isnan(row['feedback']):
        feedback = 0
    else:
        feedback = int(row['feedback'])
    #session.execute(prepared, (row['row_id'], row['session_id'], row['comment'], feedback, row['question'], row['response'], row['source_docs'], row['ts']))

In [ ]:
# This code does the data massaging for the new ledger table
import numpy as np

rows = session.execute('SELECT * FROM fiddler_chatbot_ledger')
df = rows._current_rows

def get_num_tokens(input):
    return len(input.split())

for index, row in df.iterrows():
#     prompt_tokens = get_num_tokens(row['prompt'])
#     completion_tokens = get_num_tokens(row['response'])
#     duration = np.round(np.random.normal(loc=3.0, scale=1.5),4)
      row_id = row['row_id']
#     source_docs_array = row['source_doc0'].split('Document: ')
#     #print(row['source_doc0'])
#     #print(source_docs_array)
#     #break
    
#     session.execute(f"UPDATE fiddler_chatbot_ledger SET prompt_tokens={prompt_tokens}, completion_tokens={completion_tokens}, total_tokens={prompt_tokens+completion_tokens}, \
#                       model_name='fiddler_rag_chatbot', duration={duration}, run_id='{row_id}', source_doc0='{source_docs_array[5]}', source_doc1='{source_docs_array[6]}', source_doc2='{source_docs_array[7]}' \
#                       WHERE row_id ='{row_id}'")
      sql = f"UPDATE fiddler_chatbot_ledger SET model_name='gpt-3.5-turbo' WHERE row_id='{row_id}' "
      #sql = f"DELETE FROM fiddler_chatbot_ledger WHERE row_id='{row_id}' IF model_name='' "
      print(f'SQL {index}: {sql}')
      if len(row_id) > 22:
        session.execute(sql)


In [100]:
import yaml

rows = session.execute('SELECT * FROM fiddler_chatbot_ledger')

df = rows._current_rows
df = df.sort_values(by=['ts'])
df_baseline = df.iloc[:50]
df_baseline = df_baseline.drop(columns=['ts'])
df_events = df.iloc[50:]

#dataset_info = fdl.DatasetInfo.from_dataframe(df_baseline, max_inferred_cardinality=10)

#with open('dataset.yaml', 'w') as yaml_file:
#    yaml.dump({'dataset': dataset_info.to_dict()}, yaml_file)

df_baseline.to_csv('chatbot_baseline.csv', index=False)
df_events.to_csv('chatbot_events.csv', index=False)

In [8]:
# session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")
# session.execute("DROP TABLE fiddler_chatbot_history")
# session.execute("TRUNCATE TABLE fiddler_doc_snippets_openai")
# session.execute("TRUNCATE TABLE fiddler_chatbot_history")